# W_LEDGER_D ETL Process
### EDW W_LEDGER_DS to W_LEDGER_D (Dimension SCD Type 1)
**ODI Package:** SILOS_SIL_LEDGERDIMENSION  
**IKM:** BIAPPS Oracle Incremental Update

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SILOS_SIL_LEDGERDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Drop and Create Flow Table I$_2624982_1 with change detection
DROP VIEW IF EXISTS I$_2624982_1;

CREATE OR REPLACE TEMP VIEW I$_2624982_1 AS
SELECT
  C.LEDGER_NAME,
  C.LEDGER_SHORT_NAME,
  C.LEDGER_DESC,
  C.CHART_OF_ACCOUNTS,
  C.CURRENCY_CODE,
  C.CALENDER_NAME,
  C.SLA_ACCOUNTING_METHOD_CODE,
  C.PERIOD_TYPE,
  C.LEDGER_CATEGORY_CODE,
  C.SET_ID,
  C.CREATED_BY_WID,
  C.CHANGED_BY_WID,
  C.CREATED_ON_DT,
  C.CHANGED_ON_DT,
  C.AUX1_CHANGED_ON_DT,
  C.AUX2_CHANGED_ON_DT,
  C.AUX3_CHANGED_ON_DT,
  C.AUX4_CHANGED_ON_DT,
  C.DELETE_FLG,
  C.DATASOURCE_NUM_ID,
  C.INTEGRATION_ID,
  C.TENANT_ID,
  C.X_CUSTOM,
  CASE
    -- No change: row exists and all AUX timestamps match
    WHEN T.ROW_WID IS NOT NULL
      AND (T.CHANGED_ON_DT = C.CHANGED_ON_DT OR (T.CHANGED_ON_DT IS NULL AND C.CHANGED_ON_DT IS NULL))
      AND (T.AUX1_CHANGED_ON_DT = C.AUX1_CHANGED_ON_DT OR (T.AUX1_CHANGED_ON_DT IS NULL AND C.AUX1_CHANGED_ON_DT IS NULL))
      AND (T.AUX2_CHANGED_ON_DT = C.AUX2_CHANGED_ON_DT OR (T.AUX2_CHANGED_ON_DT IS NULL AND C.AUX2_CHANGED_ON_DT IS NULL))
      AND (T.AUX3_CHANGED_ON_DT = C.AUX3_CHANGED_ON_DT OR (T.AUX3_CHANGED_ON_DT IS NULL AND C.AUX3_CHANGED_ON_DT IS NULL))
      AND (T.AUX4_CHANGED_ON_DT = C.AUX4_CHANGED_ON_DT OR (T.AUX4_CHANGED_ON_DT IS NULL AND C.AUX4_CHANGED_ON_DT IS NULL))
    THEN 'N'
    -- Update: row exists but timestamps differ
    WHEN T.ROW_WID IS NOT NULL THEN 'U'
    -- Insert: new row
    ELSE 'I'
  END AS IND_UPDATE
FROM (
  -- Source data from staging with user lookups
  SELECT
    INLINE_VIEW.LEDGER_NAME,
    INLINE_VIEW.LEDGER_SHORT_NAME,
    INLINE_VIEW.LEDGER_DESC,
    COALESCE(INLINE_VIEW.CHART_OF_ACCOUNTS, '__NOT_APPLICABLE__') AS CHART_OF_ACCOUNTS,
    COALESCE(INLINE_VIEW.CURRENCY_CODE, '__NOT_APPLICABLE__') AS CURRENCY_CODE,
    INLINE_VIEW.CALENDER_NAME,
    COALESCE(INLINE_VIEW.SLA_ACCOUNTING_METHOD_CODE, '__NOT_APPLICABLE__') AS SLA_ACCOUNTING_METHOD_CODE,
    INLINE_VIEW.PERIOD_TYPE,
    COALESCE(INLINE_VIEW.LEDGER_CATEGORY_CODE, '__NOT_APPLICABLE__') AS LEDGER_CATEGORY_CODE,
    INLINE_VIEW.SET_ID,
    -- Lookup CREATED_BY_WID from W_USER_D
    COALESCE(LKP_USER_CREATED.ROW_WID, 0) AS CREATED_BY_WID,
    -- Lookup CHANGED_BY_WID from W_USER_D
    COALESCE(LKP_USER_CHANGED.ROW_WID, 0) AS CHANGED_BY_WID,
    INLINE_VIEW.CREATED_ON_DT,
    INLINE_VIEW.CHANGED_ON_DT,
    INLINE_VIEW.AUX1_CHANGED_ON_DT,
    INLINE_VIEW.AUX2_CHANGED_ON_DT,
    INLINE_VIEW.AUX3_CHANGED_ON_DT,
    INLINE_VIEW.AUX4_CHANGED_ON_DT,
    CASE WHEN INLINE_VIEW.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
    INLINE_VIEW.DATASOURCE_NUM_ID,
    INLINE_VIEW.INTEGRATION_ID,
    INLINE_VIEW.TENANT_ID,
    INLINE_VIEW.X_CUSTOM
  FROM (
    SELECT
      SQ_W_LEDGER_DS.LEDGER_NAME,
      SQ_W_LEDGER_DS.LEDGER_SHORT_NAME,
      SQ_W_LEDGER_DS.LEDGER_DESC,
      SQ_W_LEDGER_DS.CHART_OF_ACCOUNTS,
      SQ_W_LEDGER_DS.CURRENCY_CODE,
      SQ_W_LEDGER_DS.CALENDER_NAME,
      SQ_W_LEDGER_DS.SLA_ACCOUNTING_METHOD_CODE,
      SQ_W_LEDGER_DS.PERIOD_TYPE,
      SQ_W_LEDGER_DS.LEDGER_CATEGORY_CODE,
      SQ_W_LEDGER_DS.SET_ID,
      SQ_W_LEDGER_DS.CREATED_BY_ID,
      SQ_W_LEDGER_DS.CHANGED_BY_ID,
      SQ_W_LEDGER_DS.CREATED_ON_DT,
      SQ_W_LEDGER_DS.CHANGED_ON_DT,
      SQ_W_LEDGER_DS.AUX1_CHANGED_ON_DT,
      SQ_W_LEDGER_DS.AUX2_CHANGED_ON_DT,
      SQ_W_LEDGER_DS.AUX3_CHANGED_ON_DT,
      SQ_W_LEDGER_DS.AUX4_CHANGED_ON_DT,
      SQ_W_LEDGER_DS.DELETE_FLG,
      SQ_W_LEDGER_DS.DATASOURCE_NUM_ID,
      SQ_W_LEDGER_DS.INTEGRATION_ID,
      SQ_W_LEDGER_DS.TENANT_ID,
      SQ_W_LEDGER_DS.X_CUSTOM
    FROM workspace.oracle_edw.W_LEDGER_DS SQ_W_LEDGER_DS
  ) INLINE_VIEW
  -- Lookup for CREATED_BY_WID
  LEFT JOIN workspace.oracle_edw.W_USER_D LKP_USER_CREATED
    ON INLINE_VIEW.DATASOURCE_NUM_ID = LKP_USER_CREATED.DATASOURCE_NUM_ID
    AND INLINE_VIEW.CREATED_BY_ID = LKP_USER_CREATED.INTEGRATION_ID
    AND INLINE_VIEW.CREATED_ON_DT >= LKP_USER_CREATED.EFFECTIVE_FROM_DT
    AND INLINE_VIEW.CREATED_ON_DT < LKP_USER_CREATED.EFFECTIVE_TO_DT
    AND LKP_USER_CREATED.DELETE_FLG = 'N'
  -- Lookup for CHANGED_BY_WID
  LEFT JOIN workspace.oracle_edw.W_USER_D LKP_USER_CHANGED
    ON INLINE_VIEW.DATASOURCE_NUM_ID = LKP_USER_CHANGED.DATASOURCE_NUM_ID
    AND INLINE_VIEW.CHANGED_BY_ID = LKP_USER_CHANGED.INTEGRATION_ID
    AND INLINE_VIEW.CHANGED_ON_DT >= LKP_USER_CHANGED.EFFECTIVE_FROM_DT
    AND INLINE_VIEW.CHANGED_ON_DT < LKP_USER_CHANGED.EFFECTIVE_TO_DT
    AND LKP_USER_CHANGED.DELETE_FLG = 'N'
) C
-- Outer join to target to detect changes
LEFT OUTER JOIN workspace.oracle_edw.W_LEDGER_D T
  ON C.DATASOURCE_NUM_ID = T.DATASOURCE_NUM_ID
  AND C.INTEGRATION_ID = T.INTEGRATION_ID

In [ ]:
%sql
-- Step 3: Update existing rows in W_LEDGER_D where IND_UPDATE = 'U'
MERGE INTO workspace.oracle_edw.W_LEDGER_D AS T
USING (
  SELECT * FROM I$_2624982_1 WHERE IND_UPDATE = 'U'
) S
ON T.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
  AND T.INTEGRATION_ID = S.INTEGRATION_ID
WHEN MATCHED THEN UPDATE SET
  T.LEDGER_NAME = S.LEDGER_NAME,
  T.LEDGER_SHORT_NAME = S.LEDGER_SHORT_NAME,
  T.LEDGER_DESC = S.LEDGER_DESC,
  T.CHART_OF_ACCOUNTS = S.CHART_OF_ACCOUNTS,
  T.CURRENCY_CODE = S.CURRENCY_CODE,
  T.CALENDER_NAME = S.CALENDER_NAME,
  T.SLA_ACCOUNTING_METHOD_CODE = S.SLA_ACCOUNTING_METHOD_CODE,
  T.PERIOD_TYPE = S.PERIOD_TYPE,
  T.LEDGER_CATEGORY_CODE = S.LEDGER_CATEGORY_CODE,
  T.SET_ID = S.SET_ID,
  T.CREATED_BY_WID = S.CREATED_BY_WID,
  T.CHANGED_BY_WID = S.CHANGED_BY_WID,
  T.CREATED_ON_DT = S.CREATED_ON_DT,
  T.CHANGED_ON_DT = S.CHANGED_ON_DT,
  T.AUX1_CHANGED_ON_DT = S.AUX1_CHANGED_ON_DT,
  T.AUX2_CHANGED_ON_DT = S.AUX2_CHANGED_ON_DT,
  T.AUX3_CHANGED_ON_DT = S.AUX3_CHANGED_ON_DT,
  T.AUX4_CHANGED_ON_DT = S.AUX4_CHANGED_ON_DT,
  T.DELETE_FLG = S.DELETE_FLG,
  T.TENANT_ID = S.TENANT_ID,
  T.X_CUSTOM = S.X_CUSTOM,
  T.W_UPDATE_DT = current_timestamp(),
  T.ETL_PROC_WID = 2624982

In [ ]:
%sql
-- Step 4: Insert new rows into W_LEDGER_D where IND_UPDATE = 'I'
INSERT INTO workspace.oracle_edw.W_LEDGER_D
(
  ROW_WID,
  LEDGER_NAME,
  LEDGER_SHORT_NAME,
  LEDGER_DESC,
  CHART_OF_ACCOUNTS,
  CURRENCY_CODE,
  CALENDER_NAME,
  SLA_ACCOUNTING_METHOD_CODE,
  PERIOD_TYPE,
  LEDGER_CATEGORY_CODE,
  SET_ID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID
)
SELECT
  row_number() OVER (ORDER BY INTEGRATION_ID) + COALESCE((SELECT MAX(ROW_WID) FROM workspace.oracle_edw.W_LEDGER_D), 0) AS ROW_WID,
  LEDGER_NAME,
  LEDGER_SHORT_NAME,
  LEDGER_DESC,
  CHART_OF_ACCOUNTS,
  CURRENCY_CODE,
  CALENDER_NAME,
  SLA_ACCOUNTING_METHOD_CODE,
  PERIOD_TYPE,
  LEDGER_CATEGORY_CODE,
  SET_ID,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  current_timestamp() AS W_INSERT_DT,
  current_timestamp() AS W_UPDATE_DT,
  2624982 AS ETL_PROC_WID
FROM I$_2624982_1
WHERE IND_UPDATE = 'I'

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_LEDGERDIMENSION' AS PACKAGE_NAME,
    'W_LEDGER_D' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2624982 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2624982 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_LEDGERDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop flow table (temp view)
DROP VIEW IF EXISTS I$_2624982_1;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL